<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Colab KSO Tutorials #4: Upload frames to Zooniverse</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Sep 5th, 2022</h5>

# Set up and requirements

## Install kso_data_management and its requirements

In [ ]:
# Fix issue with panoptes and libmagic
!apt-get update && apt-get install -y libmagic-dev

# Clone koster_data_management repo
!git clone --recurse-submodules -b dev https://github.com/ocean-data-factory-sweden/koster_data_management.git
!pip install -r koster_data_management/requirements.txt

# Restart the session to load the latest packages
exit()

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
os.chdir("koster_data_management/tutorials")
sys.path.append('..')

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t4_utils as t4
import kso_utils.t3_utils as t3
import kso_utils.project_utils as p_utils

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate sql and zoo project

In [ ]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

In [ ]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project)

## Retrieve info about zooniverse clips

In [ ]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project = project, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects", "classifications", "workflows"])

# Select the species of interest

In [ ]:
species_i = t4.choose_species(db_info_dict)

### Get the frames

In [ ]:
frames_to_upload_df = t4.get_frames(species_names = species_i.value, 
                                    db_path = db_info_dict["db_path"],
                                    zoo_info_dict = zoo_info_dict,
                                    server_dict = db_info_dict,
                                    project = project,
                                    n_frames_subject = 3,
                                    subsample_up_to = 3)

## List available frames to upload to Zooniverse

### Frame modifications

In [ ]:
# Work in progress
frame_modification = t3.clip_modification_widget()
frame_modification

clip_modification_widget(children=(IntText(value=0, description='Number of modifications:', style=DescriptionS…

In [ ]:
frames_to_upload_df

FileChooser(path='.', filename='', title='HTML(value='<b>Choose location to store frames</b>')', show_hidden='…

In [ ]:
# Modify the clips
frames_ready_df = t4.modify_frames(frames_to_upload_df = frames_to_upload_df.df.reset_index(drop=True), 
                                 species_i = species_i.value,
                                 modification_details = frame_modification.checks,
                                 project = project)

AttributeError: ignored

In [ ]:
# Review the size of the modified clips
t4.check_frame_size(frame_paths = frames_ready_df["modif_frame_path"].unique())

### Preview the frames

In [ ]:
# Compare the original and modified clips
t4.compare_frames(df = frames_ready_df)

### Set Zooniverse metadata

In [ ]:
upload_to_zoo = t4.set_zoo_metadata(df = frames_ready_df, species_list = species_i.value,
                                   project = project, db_info_dict = db_info_dict)

#upload_to_zoo = upload_to_zoo.fillna("unknown")

### Upload frames to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t4.upload_frames_to_zooniverse(upload_to_zoo = upload_to_zoo,
                              species_list = species_i.value,
                              db_info_dict = db_info_dict,
                              project = project)
#END